In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import os

In [4]:
USD_TO_SAR_RATE = 3.75

def convert_usd_to_sar(amount_usd):
    return amount_usd * USD_TO_SAR_RATE

In [10]:
df = pd.read_excel('../data/economic_burden/Early Diabetes Screening Data.xlsx')
df

FileNotFoundError: [Errno 2] No such file or directory: '../data/economic_burden/Early Diabetes Screening Data.xlsx'

In [9]:
possible_files = [
    "Early Diabetes Screening Data.xlsx",
    "Early Diabetes Screening Data.xlsx - اسم مجموعة البيانات.csv"
]

screening_metrics = {}

# Try to find and load the file
loaded = False
for file_path in possible_files:
    if os.path.exists(file_path):
        print(f"   [Info] File found: {file_path}")
        try:
            # Attempt to read based on extension or content
            if file_path.endswith('.csv'):
                df_raw = pd.read_csv(file_path)
            else:
                df_raw = pd.read_excel(file_path)
            
            # Data Extraction Logic (Searching for keywords)
            # This ensures we get the value regardless of row position
            metrics_map = {
                'عدد المفحوصين للبرنامج': 'Total_Screened',
                'المصابين بمرحلة ما قبل السكري': 'Pre_Diabetes',
                'المصابين بالسكري': 'Diabetes_Detected'
            }
            
            # Clean column names
            df_raw.columns = df_raw.columns.str.strip()
            
            for ar_key, en_key in metrics_map.items():
                # Search in the first column (index 0)
                # We cast to string to avoid errors if data is mixed
                row = df_raw[df_raw.iloc[:, 0].astype(str).str.contains(ar_key, na=False)]
                
                if not row.empty:
                    val = row.iloc[0, 1] # Value is in the second column
                    # Clean number string (remove commas)
                    clean_val = float(str(val).replace(',', '').replace('nan', '0'))
                    screening_metrics[en_key] = clean_val
                else:
                    screening_metrics[en_key] = 0.0
            
            loaded = True
            break # Stop if successful
        except Exception as e:
            print(f"   [Warning] Failed to read {file_path}: {e}")

if not loaded:
    print("   [Error] Could not load screening data file. Using placeholder values.")
    screening_metrics = {'Total_Screened': 722714, 'Pre_Diabetes': 48472, 'Diabetes_Detected': 32912}

   [Error] Could not load screening data file. Using placeholder values.
